In [1]:
from sklearn.datasets import load_breast_cancer
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.python.keras.layers import Layer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import datasets

demo_data = load_breast_cancer()

In [3]:
from sklearn.model_selection import train_test_split

x_train_all, x_test, y_train_all, y_test = train_test_split(demo_data.data, demo_data.target, random_state = 7)

x_train, x_valid, y_train, y_valid = train_test_split(x_train_all, y_train_all, random_state = 11)

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

Functional API

In [7]:
input_wide = tf.keras.layers.Input(shape=[15], name='input_wide')
input_deep = tf.keras.layers.Input(shape=[15], name='input_deep')
hidden1 = tf.keras.layers.Dense(30, activation='relu')(input_deep)
hidden2 = tf.keras.layers.Dense(30, activation='relu')(hidden1)
concat = tf.keras.layers.concatenate([input_wide, hidden2])
output = tf.keras.layers.Dense(1, activation='sigmoid')(concat)

model = tf.keras.models.Model(inputs = [input_wide, input_deep], outputs = [output])

Model Subclassing

In [8]:
class WideDeepModel(tf.keras.models.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = tf.keras.layers.Dense(30, activation='relu')
        self.hidden2 = tf.keras.layers.Dense(30, activation='relu')
        self.concat = tf.keras.layers.concatenate
        self.dense = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        input_wide, input_deep = inputs
        output_deep = self.hidden1(input_deep)
        output_deep = self.hidden2(output_deep)
        concat_input = self.concat([input_wide, output_deep])
        output = self.dense(concat_input)
        return output

In [12]:
model = WideDeepModel()

In [15]:
# 单个loss
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
callbacks = [tf.keras.callbacks.EarlyStopping(patience=5, min_delta=1e-2)]

模型训练

In [16]:
# 将训练集的前15列作为 wide 部分的输入，后15列作为 deep 部分的输入, 训练10轮
x_train_scaled_wide = x_train_scaled[:, :15]
x_train_scaled_deep = x_train_scaled[:, 15:]
x_valid_scaled_wide = x_valid_scaled[:, :15]
x_valid_scaled_deep = x_valid_scaled[:, 15:]
x_test_scaled_wide = x_test_scaled[:, :15]
x_test_scaled_deep = x_test_scaled[:, 15:]


history = model.fit([x_train_scaled_wide, x_train_scaled_deep], y_train,
                   validation_data=([x_valid_scaled_wide, x_valid_scaled_deep], y_valid),
                   epochs=10)

Epoch 1/10
10/10 [==============================] - 0s 41ms/step - loss: 0.5474 - accuracy: 0.7524 - val_loss: 0.4726 - val_accuracy: 0.8692
Epoch 2/10
10/10 [==============================] - 0s 6ms/step - loss: 0.4476 - accuracy: 0.8401 - val_loss: 0.3858 - val_accuracy: 0.9159
Epoch 3/10
10/10 [==============================] - 0s 6ms/step - loss: 0.3684 - accuracy: 0.8777 - val_loss: 0.3184 - val_accuracy: 0.9252
Epoch 4/10
10/10 [==============================] - 0s 6ms/step - loss: 0.3049 - accuracy: 0.9122 - val_loss: 0.2647 - val_accuracy: 0.9252
Epoch 5/10
10/10 [==============================] - 0s 5ms/step - loss: 0.2553 - accuracy: 0.9310 - val_loss: 0.2241 - val_accuracy: 0.9346
Epoch 6/10
10/10 [==============================] - 0s 5ms/step - loss: 0.2155 - accuracy: 0.9436 - val_loss: 0.1928 - val_accuracy: 0.9533
Epoch 7/10
10/10 [==============================] - 0s 5ms/step - loss: 0.1841 - accuracy: 0.9498 - val_loss: 0.1680 - val_accuracy: 0.9533
Epoch 8/10
10/10 [=